In [1]:
import calliope
import pandas as pd
import plotly.express as px

calliope.set_log_verbosity("INFO", include_solver_output=False)

In [2]:
model = calliope.Model('model.yaml')

[2025-04-14 20:34:03] INFO     Model: initialising
[2025-04-14 20:34:03] INFO     Model: preprocessing stage 1 (model_run)
[2025-04-14 20:34:04] INFO     Model: preprocessing stage 2 (model_data)
[2025-04-14 20:34:04] INFO     Model: preprocessing complete


In [3]:
model.inputs

<xarray.Dataset> Size: 114kB
Dimensions:                 (costs: 1, techs: 4, nodes: 2, carriers: 2,
                             timesteps: 744)
Coordinates:
  * costs                   (costs) object 8B 'monetary'
  * techs                   (techs) object 32B 'aluminium_demand' ... 'fuel_s...
  * carriers                (carriers) object 16B 'aluminium' 'fuel'
  * nodes                   (nodes) <U11 88B 'Iceland' 'Netherlands'
  * timesteps               (timesteps) datetime64[ns] 6kB 2005-01-01 ... 200...
Data variables: (12/24)
    bigM                    float64 8B 1e+06
    objective_cost_weights  (costs) float64 8B 1.0
    base_tech               (techs) object 32B 'demand' 'supply' ... 'supply'
    carrier                 (nodes, techs) object 64B nan 'aluminium' ... nan
    carrier_out             (nodes, techs, carriers) float64 128B nan ... nan
    color                   (techs) object 32B '#072486' '#F9CF22' ... '#E37A72'
    ...                      ...
    sink_use_equals         (timesteps, nodes, techs) float64 48kB nan ... nan
    source_use_max          (timesteps, nodes, techs) float64 48kB nan ... nan
    definition_matrix       (nodes, techs, carriers) bool 16B False ... False
    distance                (techs) float64 32B nan nan 2.021e+03 nan
    timestep_resolution     (timesteps) float64 6kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
    timestep_weights        (timesteps) float64 6kB 1.0 1.0 1.0 ... 1.0 1.0 1.0
Attributes:
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev6
    applied_overrides:             
    scenario:                      None
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    allow_operate_mode:            1
    name:                          main model
    timestamp_model_creation:      1744655643.699526

In [4]:
model.inputs.flow_cap_max.to_series().dropna()

techs
aluminium_supply                inf
aluminium_transport_tech    20000.0
fuel_supply                     inf
Name: flow_cap_max, dtype: float64

In [5]:
model.inputs.sink_use_equals.sum(
    "timesteps", min_count=1, skipna=True
).to_series().dropna()

nodes        techs           
Netherlands  aluminium_demand    743999256.0
Name: sink_use_equals, dtype: float64

In [6]:
model.build(force=True)
model.solve()

[2025-04-14 20:34:04] INFO     Model: backend build starting
[2025-04-14 20:34:04] INFO     Math preprocessing | added file 'plan'.
[2025-04-14 20:34:04] INFO     Math preprocessing | added file 'custom_constraints_state.yml'.
[2025-04-14 20:34:04] INFO     Math preprocessing | validated math against schema.
[2025-04-14 20:34:04] INFO     Optimisation Model | parameters | Generated.
[2025-04-14 20:34:06] INFO     Optimisation Model | Validated math strings.
[2025-04-14 20:34:06] INFO     Optimisation Model | variables | Generated.
[2025-04-14 20:34:07] INFO     Optimisation Model | global_expressions | Generated.
[2025-04-14 20:34:08] INFO     Optimisation Model | constraints | Generated.
[2025-04-14 20:34:08] INFO     Optimisation Model | piecewise_constraints | Generated.
[2025-04-14 20:34:08] INFO     Optimisation Model | objectives | Generated.
[2025-04-14 20:34:08] INFO     Model: backend build complete
[2025-04-14 20:34:08] INFO     Optimisation model | starting model in plan mod

In [7]:
model.results

<xarray.Dataset> Size: 650kB
Dimensions:                     (nodes: 2, techs: 4, carriers: 2,
                                 timesteps: 744, costs: 1)
Coordinates:
  * techs                       (techs) object 32B 'aluminium_demand' ... 'fu...
  * nodes                       (nodes) <U11 88B 'Iceland' 'Netherlands'
  * carriers                    (carriers) <U9 72B 'aluminium' 'fuel'
  * timesteps                   (timesteps) datetime64[ns] 6kB 2005-01-01 ......
  * costs                       (costs) object 8B 'monetary'
Data variables: (12/21)
    flow_cap                    (nodes, techs, carriers) float64 128B nan ......
    link_flow_cap               (techs) float64 32B nan nan 2e+04 nan
    flow_out                    (nodes, techs, carriers, timesteps) float64 95kB ...
    flow_in                     (nodes, techs, carriers, timesteps) float64 95kB ...
    source_use                  (nodes, techs, timesteps) float64 48kB nan .....
    source_cap                  (nodes, techs) float64 64B nan 2e+04 ... nan nan
    ...                          ...
    cost_investment_annualised  (nodes, techs, costs) float64 64B nan ... nan
    cost                        (nodes, techs, costs) float64 64B nan ... nan
    capacity_factor             (nodes, techs, carriers, timesteps) float64 95kB ...
    systemwide_capacity_factor  (techs, carriers) float64 64B 0.0 0.0 ... 0.0
    systemwide_levelised_cost   (carriers, techs, costs) float64 64B nan ... nan
    total_levelised_cost        (carriers, costs) float64 16B 0.0 nan
Attributes: (12/13)
    termination_condition:         optimal
    calliope_version_defined:      0.7.0
    calliope_version_initialised:  0.7.0.dev6
    applied_overrides:             
    scenario:                      None
    defaults:                      {'bigM': 1000000000.0, 'objective_cost_wei...
    ...                            ...
    name:                          main model
    timestamp_model_creation:      1744655643.699526
    timestamp_build_start:         1744655644.434678
    timestamp_build_complete:      1744655648.262941
    timestamp_solve_start:         1744655648.262941
    timestamp_solve_complete:      1744655649.000479

In [8]:
costs = model.results.cost.to_series().dropna()
costs.head()

nodes    techs             costs   
Iceland  aluminium_supply  monetary    0.0
         fuel_supply       monetary    0.0
Name: cost, dtype: float64

In [9]:
lcoes = model.results.systemwide_levelised_cost.to_series().dropna()
lcoes

carriers   techs             costs   
aluminium  aluminium_supply  monetary    0.0
Name: systemwide_levelised_cost, dtype: float64

In [10]:
colors = model.inputs.color.to_series().to_dict()

In [11]:
model.to_netcdf("main_model.nc")